In [1]:
"""
批量绘制SWC神经元形态图 - 按编号范围着色
"""

import os
import navis
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import warnings
import re

def get_neuron_number(filename):
    """从文件名中提取神经元编号"""
    match = re.search(r'neuron_(\d+)', filename)
    if match:
        return int(match.group(1))
    return None

def get_color_by_number(neuron_number):
    """根据神经元编号返回颜色"""
    if 1 <= neuron_number <= 1800:
        return '#FF0000'  # 红色
    elif 1801 <= neuron_number <= 2600:
        return '#0000FF'  # 蓝色
    elif 2601 <= neuron_number <= 3000:
        return '#00FF00'  # 绿色
    else:
        return '#808080'  # 灰色（默认）

def plot_neuron_clean(swc_path, output_path, figsize=(8, 8), dpi=150, 
                     view_angles=(30, 30), linewidth=1.5):
    """
    绘制单个神经元的简洁形态图（无坐标轴、网格等）
    
    参数:
    - swc_path: SWC文件路径
    - output_path: 输出图片路径
    - figsize: 图片大小
    - dpi: 图片分辨率
    - view_angles: 3D视角 (elevation, azimuth)
    - linewidth: 线条粗细
    """
    try:
        # 读取SWC文件
        neuron = navis.read_swc(swc_path)
        
        # 获取神经元编号并确定颜色
        filename = os.path.basename(swc_path)
        neuron_number = get_neuron_number(filename)
        color = get_color_by_number(neuron_number) if neuron_number else '#808080'
        
        # 创建图形
        fig = plt.figure(figsize=figsize, facecolor='white')
        ax = fig.add_subplot(111, projection='3d')
        
        # 绘制神经元
        navis.plot3d(
            neuron,
            ax=ax,
            color=color,
            linewidth=linewidth,
            connectors=False
        )
        
        # 设置视角
        ax.view_init(elev=view_angles[0], azim=view_angles[1])
        
        # 移除所有坐标轴、网格、标签等
        ax.set_axis_off()
        ax.grid(False)
        
        # 移除背景
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('none')
        ax.yaxis.pane.set_edgecolor('none')
        ax.zaxis.pane.set_edgecolor('none')
        
        # 调整边距，使神经元充满画布
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
        # 保存图片
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', 
                   facecolor='white', pad_inches=0)
        plt.close(fig)
        
        return True, None
        
    except Exception as e:
        plt.close('all')
        return False, str(e)


def batch_plot_neurons_by_range(input_dir, output_dir, figsize=(8, 8), dpi=150,
                                view_angles=(30, 30), linewidth=1.5):
    """
    批量绘制神经元，按编号范围着色
    
    参数:
    - input_dir: 输入SWC文件夹路径
    - output_dir: 输出图片文件夹路径
    - figsize: 图片大小
    - dpi: 图片分辨率
    - view_angles: 3D视角
    - linewidth: 线条粗细
    """
    
    # 创建路径对象
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    # 创建输出文件夹（如果不存在）
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 获取所有SWC文件并按编号排序
    swc_files = list(input_path.glob('neuron_*.swc'))
    
    # 按神经元编号排序
    swc_files.sort(key=lambda x: get_neuron_number(x.name) or 0)
    
    if not swc_files:
        print(f"警告：在 {input_dir} 中没有找到neuron_*.swc文件")
        return
    
    print(f"找到 {len(swc_files)} 个SWC文件")
    
    # 记录处理结果
    results = {
        'successful': [],
        'failed': [],
        'color_stats': {
            'red': 0,    # 1-1800
            'blue': 0,   # 1801-2600
            'green': 0   # 2601-3000
        }
    }
    
    # 处理每个SWC文件
    for swc_file in tqdm(swc_files, desc="绘制神经元"):
        filename = swc_file.stem  # 不包含扩展名的文件名
        neuron_number = get_neuron_number(swc_file.name)
        
        # 统计颜色分布
        if neuron_number:
            if 1 <= neuron_number <= 1800:
                results['color_stats']['red'] += 1
            elif 1801 <= neuron_number <= 2600:
                results['color_stats']['blue'] += 1
            elif 2601 <= neuron_number <= 3000:
                results['color_stats']['green'] += 1
        
        # 绘制神经元
        output_file = output_path / f'{filename}.png'
        success, error = plot_neuron_clean(
            swc_file, output_file, 
            figsize=figsize, dpi=dpi, 
            view_angles=view_angles, linewidth=linewidth
        )
        
        if success:
            results['successful'].append(filename)
        else:
            results['failed'].append((filename, error))
    
    # 打印结果摘要
    print("\n" + "="*60)
    print("处理完成")
    print("="*60)
    print(f"成功绘制: {len(results['successful'])} 个")
    print(f"\n颜色分布统计:")
    print(f"  红色 (neuron_1-1800):     {results['color_stats']['red']} 个")
    print(f"  蓝色 (neuron_1801-2600):  {results['color_stats']['blue']} 个")
    print(f"  绿色 (neuron_2601-3000):  {results['color_stats']['green']} 个")
    
    if results['failed']:
        print(f"\n处理失败: {len(results['failed'])} 个")
        for name, error in results['failed'][:5]:  # 只显示前5个错误
            print(f"  - {name}: {error}")
        if len(results['failed']) > 5:
            print(f"  ... 还有 {len(results['failed']) - 5} 个错误")
    
    print("="*60)
    
    return results


# 主执行函数
def main():
    """主函数 - 执行批量绘制"""
    
    # 设置路径
    input_dir = "/Users/apple/Desktop/KLin_Group/Project_2024/data/Morpho_data/dataset/Simulation/v4/swc_files"
    output_dir = "/Users/apple/Desktop/KLin_Group/Project_2024/data/Morpho_data/dataset/Simulation/v4/neuron_plot"
    
    # 执行批量绘制
    results = batch_plot_neurons_by_range(
        input_dir=input_dir,
        output_dir=output_dir,
        figsize=(8, 8),       # 图片大小
        dpi=150,              # 图片质量
        view_angles=(30, 30), # 3D视角 (elevation, azimuth)
        linewidth=1.5         # 线条粗细
    )
    
    print(f"\n图片已保存到: {output_dir}")
    
    return results


# 如果你想直接运行
if __name__ == "__main__":
    # 设置matplotlib后端（避免GUI问题）
    import matplotlib
    matplotlib.use('Agg')
    
    # 忽略某些警告
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore')
    
    # 运行主函数
    results = main()

找到 3000 个SWC文件


绘制神经元: 100%|██████████| 3000/3000 [01:02<00:00, 47.99it/s]


处理完成
成功绘制: 0 个

颜色分布统计:
  红色 (neuron_1-1800):     1800 个
  蓝色 (neuron_1801-2600):  800 个
  绿色 (neuron_2601-3000):  400 个

处理失败: 3000 个
  - neuron_1: The 'ax' argument is invalid for plotly backend. Valid arguments are: alpha, cn_alpha, cn_colors, cn_layout, cn_mesh_colors, cn_size, color, color_by, connectors, connectors_only, dps_scale_vec, fig, fig_autosize, height, hover_id, hover_name, inline, legend, legend_group, legend_orientation, linestyle, linewidth, norm_global, palette, radius, scatter_kws, shade_by, smax, smin, soma, title, vmax, vmin, volume_legend, width
  - neuron_2: The 'ax' argument is invalid for plotly backend. Valid arguments are: alpha, cn_alpha, cn_colors, cn_layout, cn_mesh_colors, cn_size, color, color_by, connectors, connectors_only, dps_scale_vec, fig, fig_autosize, height, hover_id, hover_name, inline, legend, legend_group, legend_orientation, linestyle, linewidth, norm_global, palette, radius, scatter_kws, shade_by, smax, smin, soma, title, vmax, vmin, vol

In [3]:
"""
批量绘制SWC神经元形态图 - 按编号范围着色
"""

import os
import navis
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import warnings
import re

# 强制使用matplotlib后端
navis.set_pbars(hide=True)
navis.set_default_connector_colors({})

def get_neuron_number(filename):
    """从文件名中提取神经元编号"""
    match = re.search(r'neuron_(\d+)', filename)
    if match:
        return int(match.group(1))
    return None

def get_color_by_number(neuron_number):
    """根据神经元编号返回颜色"""
    if 1 <= neuron_number <= 1800:
        return '#FF0000'  # 红色
    elif 1801 <= neuron_number <= 2600:
        return '#0000FF'  # 蓝色
    elif 2601 <= neuron_number <= 3000:
        return '#00FF00'  # 绿色
    else:
        return '#808080'  # 灰色（默认）

def plot_neuron_clean(swc_path, output_path, figsize=(8, 8), dpi=150, 
                     view_angles=(30, 30), linewidth=1.5):
    """
    绘制单个神经元的简洁形态图（无坐标轴、网格等）
    
    参数:
    - swc_path: SWC文件路径
    - output_path: 输出图片路径
    - figsize: 图片大小
    - dpi: 图片分辨率
    - view_angles: 3D视角 (elevation, azimuth)
    - linewidth: 线条粗细
    """
    try:
        # 读取SWC文件
        neuron = navis.read_swc(swc_path)
        
        # 获取神经元编号并确定颜色
        filename = os.path.basename(swc_path)
        neuron_number = get_neuron_number(filename)
        color = get_color_by_number(neuron_number) if neuron_number else '#808080'
        
        # 创建图形
        fig = plt.figure(figsize=figsize, facecolor='white')
        ax = fig.add_subplot(111, projection='3d')
        
        # 使用navis的plot2d函数的底层实现来绘制3D
        # 直接操作神经元的节点和边
        nodes = neuron.nodes
        
        # 绘制所有的线段
        for _, row in neuron.nodes.iterrows():
            if row.parent_id >= 0:  # 如果有父节点
                parent = neuron.nodes[neuron.nodes.node_id == row.parent_id]
                if not parent.empty:
                    parent = parent.iloc[0]
                    # 绘制从父节点到当前节点的线段
                    ax.plot3D(
                        [parent.x, row.x],
                        [parent.y, row.y],
                        [parent.z, row.z],
                        color=color,
                        linewidth=linewidth,
                        alpha=0.8
                    )
        
        # 设置视角
        ax.view_init(elev=view_angles[0], azim=view_angles[1])
        
        # 移除所有坐标轴、网格、标签等
        ax.set_axis_off()
        ax.grid(False)
        
        # 移除背景
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis.pane.set_edgecolor('none')
        ax.yaxis.pane.set_edgecolor('none')
        ax.zaxis.pane.set_edgecolor('none')
        
        # 设置相等的坐标轴比例
        max_range = max(
            nodes.x.max() - nodes.x.min(),
            nodes.y.max() - nodes.y.min(),
            nodes.z.max() - nodes.z.min()
        ) / 2.0
        
        mid_x = (nodes.x.max() + nodes.x.min()) / 2.0
        mid_y = (nodes.y.max() + nodes.y.min()) / 2.0
        mid_z = (nodes.z.max() + nodes.z.min()) / 2.0
        
        ax.set_xlim(mid_x - max_range, mid_x + max_range)
        ax.set_ylim(mid_y - max_range, mid_y + max_range)
        ax.set_zlim(mid_z - max_range, mid_z + max_range)
        
        # 调整边距，使神经元充满画布
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
        # 保存图片
        plt.savefig(output_path, dpi=dpi, bbox_inches='tight', 
                   facecolor='white', pad_inches=0)
        plt.close(fig)
        
        return True, None
        
    except Exception as e:
        plt.close('all')
        return False, str(e)


def batch_plot_neurons_by_range(input_dir, output_dir, figsize=(8, 8), dpi=150,
                                view_angles=(30, 30), linewidth=1.5):
    """
    批量绘制神经元，按编号范围着色
    
    参数:
    - input_dir: 输入SWC文件夹路径
    - output_dir: 输出图片文件夹路径
    - figsize: 图片大小
    - dpi: 图片分辨率
    - view_angles: 3D视角
    - linewidth: 线条粗细
    """
    
    # 创建路径对象
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    # 创建输出文件夹（如果不存在）
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 获取所有SWC文件并按编号排序
    swc_files = list(input_path.glob('neuron_*.swc'))
    
    # 按神经元编号排序
    swc_files.sort(key=lambda x: get_neuron_number(x.name) or 0)
    
    if not swc_files:
        print(f"警告：在 {input_dir} 中没有找到neuron_*.swc文件")
        return
    
    print(f"找到 {len(swc_files)} 个SWC文件")
    
    # 记录处理结果
    results = {
        'successful': [],
        'failed': [],
        'color_stats': {
            'red': 0,    # 1-1800
            'blue': 0,   # 1801-2600
            'green': 0   # 2601-3000
        }
    }
    
    # 处理每个SWC文件
    for swc_file in tqdm(swc_files, desc="绘制神经元"):
        filename = swc_file.stem  # 不包含扩展名的文件名
        neuron_number = get_neuron_number(swc_file.name)
        
        # 统计颜色分布
        if neuron_number:
            if 1 <= neuron_number <= 1800:
                results['color_stats']['red'] += 1
            elif 1801 <= neuron_number <= 2600:
                results['color_stats']['blue'] += 1
            elif 2601 <= neuron_number <= 3000:
                results['color_stats']['green'] += 1
        
        # 绘制神经元
        output_file = output_path / f'{filename}.png'
        success, error = plot_neuron_clean(
            swc_file, output_file, 
            figsize=figsize, dpi=dpi, 
            view_angles=view_angles, linewidth=linewidth
        )
        
        if success:
            results['successful'].append(filename)
        else:
            results['failed'].append((filename, error))
    
    # 打印结果摘要
    print("\n" + "="*60)
    print("处理完成")
    print("="*60)
    print(f"成功绘制: {len(results['successful'])} 个")
    print(f"\n颜色分布统计:")
    print(f"  红色 (neuron_1-1800):     {results['color_stats']['red']} 个")
    print(f"  蓝色 (neuron_1801-2600):  {results['color_stats']['blue']} 个")
    print(f"  绿色 (neuron_2601-3000):  {results['color_stats']['green']} 个")
    
    if results['failed']:
        print(f"\n处理失败: {len(results['failed'])} 个")
        for name, error in results['failed'][:5]:  # 只显示前5个错误
            print(f"  - {name}: {error}")
        if len(results['failed']) > 5:
            print(f"  ... 还有 {len(results['failed']) - 5} 个错误")
    
    print("="*60)
    
    return results


# 主执行函数
def main():
    """主函数 - 执行批量绘制"""
    
    # 设置路径
    input_dir = "/Users/apple/Desktop/KLin_Group/Project_2024/data/Morpho_data/dataset/Simulation/v4/swc_files"
    output_dir = "/Users/apple/Desktop/KLin_Group/Project_2024/data/Morpho_data/dataset/Simulation/v4/neuron_plot"
    
    # 执行批量绘制
    results = batch_plot_neurons_by_range(
        input_dir=input_dir,
        output_dir=output_dir,
        figsize=(8, 8),       # 图片大小
        dpi=150,              # 图片质量
        view_angles=(30, 30), # 3D视角 (elevation, azimuth)
        linewidth=1.5         # 线条粗细
    )
    
    print(f"\n图片已保存到: {output_dir}")
    
    return results


# 如果你想直接运行
if __name__ == "__main__":
    # 设置matplotlib后端（避免GUI问题）
    import matplotlib
    matplotlib.use('Agg')
    
    # 忽略某些警告
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore')
    
    # 运行主函数
    results = main()

找到 3000 个SWC文件


绘制神经元: 100%|██████████| 3000/3000 [08:52<00:00,  5.63it/s]


处理完成
成功绘制: 3000 个

颜色分布统计:
  红色 (neuron_1-1800):     1800 个
  蓝色 (neuron_1801-2600):  800 个
  绿色 (neuron_2601-3000):  400 个

图片已保存到: /Users/apple/Desktop/KLin_Group/Project_2024/data/Morpho_data/dataset/Simulation/v4/neuron_plot
